In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import cm
import os
from scipy.stats import skew

In [2]:
df_i_next = pd.read_spss('data/caspian1 data.sav',convert_categoricals=True)


In [3]:
def preprocess(df_org):

    df = df_org.copy()

    # Filter age
    df = df[(df["age"] >= 7) & (df["age"] <= 18)]
    # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
    df['height'] = pd.to_numeric(df['height'], errors='coerce')
    df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
    df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
    # Remove null tuples
    records_with_nulls = df[
        df[["weight", "height", "sex", "age"]].isna().any(axis=1)
    ]
    df = df.dropna(subset=["height", "weight", "sex"])
    print(
        f"Number of records with NaN value in weight or height: {len(records_with_nulls)}"
    )   

    return df
df_i_next = preprocess(df_i_next)


Number of records with NaN value in weight or height: 132


In [4]:
# Group by Age and Sex, then calculate summary statistics (mean, for example)
grouped = df_i_next.groupby(['age', 'sex'])

for (age, sex), group in grouped:
    print(f"Group for Age: {age}, Sex: {sex}")
    print(len(group))
    print()


Group for Age: 7.0, Sex: Female
656

Group for Age: 7.0, Sex: Male
664

Group for Age: 8.0, Sex: Female
729

Group for Age: 8.0, Sex: Male
766

Group for Age: 9.0, Sex: Female
688

Group for Age: 9.0, Sex: Male
744

Group for Age: 10.0, Sex: Female
791

Group for Age: 10.0, Sex: Male
841

Group for Age: 11.0, Sex: Female
816

Group for Age: 11.0, Sex: Male
932

Group for Age: 12.0, Sex: Female
1073

Group for Age: 12.0, Sex: Male
1031

Group for Age: 13.0, Sex: Female
900

Group for Age: 13.0, Sex: Male
1028

Group for Age: 14.0, Sex: Female
1025

Group for Age: 14.0, Sex: Male
1222

Group for Age: 15.0, Sex: Female
1046

Group for Age: 15.0, Sex: Male
1066

Group for Age: 16.0, Sex: Female
1029

Group for Age: 16.0, Sex: Male
985

Group for Age: 17.0, Sex: Female
664

Group for Age: 17.0, Sex: Male
733

Group for Age: 18.0, Sex: Female
287

Group for Age: 18.0, Sex: Male
299



/tmp/ipykernel_37927/1076703598.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_i_next.groupby(['age', 'sex'])


In [5]:
def calculate_z_score(value, L, M, S):
    """
    Calculate the z-score using the LMS method.

    Parameters:
    - value: Observed value (e.g., height, weight, BMI).
    - L: Lambda (skewness parameter).
    - M: Mu (median or central tendency).
    - S: Sigma (coefficient of variation).

    Returns:
    - z_score: Standardized z-score.
    """
    if L == 0:
        z_score = (value / M - 1) / S
    else:
        z_score = (np.power(value / M, L) - 1) / (L * S)
    return z_score

def apply_z_score(group, params):
    # Merge params (Lambda, Median, Sigma) with the group
    group = group.merge(params, on='sex', how='left')
    
    # Apply the z-score calculation row-wise within each group
    group['Z-Score'] = group.apply(
        lambda row: calculate_z_score(row['height'], L=row['Lambda'], M=row['Median'], S=row['Sigma']), axis=1
    )
    return group

def calculate_params(group, feature):
    # Calculate Median (M)
    M = group[f'{feature}'].median()

    # Calculate Lambda (L) - Skewness
    L = skew(group[f'{feature}'])

    # Calculate Sigma (S) - Coefficient of Variation (std / mean)
    mean = group[f'{feature}'].mean()
    std_dev = group[f'{feature}'].std()
    S = std_dev / mean if mean != 0 else None  # Avoid division by zero

    return pd.Series({'Lambda': L, 'Median': M, 'Sigma': S})

# Apply the function to each group, passing the feature parameter dynamically
params = grouped.apply(lambda group: calculate_params(group, feature='height')).reset_index()

# Now, apply this to each group, passing the params DataFrame
grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params))

# Display the results
print(grouped_with_z_score)

/tmp/ipykernel_37927/857306185.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature='height')).reset_index()


                     id     code     sex    univer  state  province district  \
age  sex                                                                       
7.0  Female 0     300.0  30293.0  Female  Oroomieh    3.0       2.0    Urban   
            1     300.0  30293.0  Female  Oroomieh    3.0       2.0    Urban   
            2     300.0  30293.0  Female  Oroomieh    3.0       2.0    Urban   
            3     300.0  30293.0  Female  Oroomieh    3.0       2.0    Urban   
            4     300.0  30293.0  Female  Oroomieh    3.0       2.0    Urban   
...                 ...      ...     ...       ...    ...       ...      ...   
18.0 Male   3583  751.0  10746.0    Male      Arak    1.0       1.0    Rural   
            3584  751.0  10746.0    Male      Arak    1.0       1.0    Rural   
            3585  751.0  10746.0    Male      Arak    1.0       1.0    Rural   
            3586  751.0  10746.0    Male      Arak    1.0       1.0    Rural   
            3587  751.0  10746.0    Male

/tmp/ipykernel_37927/857306185.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params))


In [6]:
percentiles = [5, 50, 95]  # Define percentiles
Z_scores = [norm.ppf(p / 100) for p in percentiles]  # Convert percentiles to Z-scores

for Z in Z_scores:
    grouped_with_z_score[f'P{int(norm.cdf(Z)*100)}'] = params_df.apply(
        lambda row: row['Median'] * (1 + row['Lambda'] * row['Sigma'] * Z)**(1/row['Lambda'])
        if row['Lambda'] != 0 else
        row['Median'] * np.exp(row['Sigma'] * Z), axis=1
    )


NameError: name 'norm' is not defined